# Annex 96 Common Exercise 1 – Quick Start Notebook

This notebook provides a minimal end-to-end setup for:

1. Installing `CityLearn==2.5.0`
2. Cloning the CE1 GitHub repository
3. Understanding the TX & VT datasets
4. Visualizing the daily reference district load
5. Running a simple baseline battery RBC controller
6. Evaluating KPIs

If you run into issues, please reach out on the Annex 96 Slack (Common Exercise channel).


## 1. Install CityLearn



In [1]:
!pip install "CityLearn==2.5.0"




## 2. Clone the Common Exercise 1 Repository

This repository contains:

- Pre-configured scenarios for **Texas (TX)** and **Vermont (VT)**  
- The `schema_clean.json` files for the 25 homes  
- Pre-computed reference district load profiles  
- Example notebooks (including this Quick Start)

> Replace the placeholder URL with the actual CE1 GitHub link.



## 3. Dataset Overview (TX vs VT)

The Common Exercise uses two scenarios:

### 🔵 Texas (TX) – cooling dominated  
- Training: **August**  
- Testing: **September**  
- Higher solar generation, strong cooling loads.

### 🔵 Vermont (VT) – heating dominated  
- Training: **January**  
- Testing: **February**  
- High heating loads, low PV in winter.

Each climate folder contains:

- `schema.json`  
- 25 ResStock buildings  
- Pre-computed district reference load  
- All data handled automatically by CityLearn




In [1]:
from pathlib import Path
import os

# Where the notebook actually lives
NOTEBOOK_DIR = Path(os.getcwd())
print("Notebook directory:", NOTEBOOK_DIR)

# Go one level up to the repo root
BASE_DIR = NOTEBOOK_DIR.parent
print("Repository root:", BASE_DIR)

# Choose climate: TX or VT
CLIMATE = "VT"  # or "VT"

DATASET_DIR = BASE_DIR / "data" / "datasets" / f"annex96_ce1_{CLIMATE.lower()}_neighborhood"
SCHEMA_PATH = DATASET_DIR / "schema.json"

print("\nClimate:", CLIMATE)
print("Dataset directory:", DATASET_DIR, " | exists:", DATASET_DIR.exists())
print("Schema path:", SCHEMA_PATH, " | exists:", SCHEMA_PATH.exists())


Notebook directory: /home/ava_mohammadi/PhD/annex96_ce1/annex96_common_exercise_1/notebooks
Repository root: /home/ava_mohammadi/PhD/annex96_ce1/annex96_common_exercise_1

Climate: VT
Dataset directory: /home/ava_mohammadi/PhD/annex96_ce1/annex96_common_exercise_1/data/datasets/annex96_ce1_vt_neighborhood  | exists: True
Schema path: /home/ava_mohammadi/PhD/annex96_ce1/annex96_common_exercise_1/data/datasets/annex96_ce1_vt_neighborhood/schema.json  | exists: True


## 4. Reference District Load

Each day, the Annex 96 CE defines a **constant district load target** equal to the average aggregated demand of all buildings for the next day.

Controllers attempt to track this reference profile.

Below is the reference load plot for the selected climate (TX or VT).


## 5. Run a Simple Battery RBC Controller

This baseline:

- Uses the built-in `BasicRBC` controller  
- Controls all buildings through a **central agent**  
- Produces reference KPIs to compare against learning-based methods  


In [2]:
import sys
from pathlib import Path

# Show current paths once (optional, for debugging)
print("Before cleanup:")
for p in sys.path:
    print("  ", p)

# Remove any paths that point to your local dev CityLearn repo
CANDIDATES = ["Occupant_Thermostat_Int", "CityLearn"]

sys.path = [
    p for p in sys.path
    if not any(c in (p or "") for c in CANDIDATES)
]

print("\nAfter cleanup:")
for p in sys.path:
    print("  ", p)


Before cleanup:
   /home/ava_mohammadi/PhD/CityLearn/Occupant_Thermostat_Int
   /home/ava_mohammadi/PhD/annex96_ce1/annex96_common_exercise_1/notebooks
   /home/ava_mohammadi/miniconda3/envs/annex96_ce1/lib/python310.zip
   /home/ava_mohammadi/miniconda3/envs/annex96_ce1/lib/python3.10
   /home/ava_mohammadi/miniconda3/envs/annex96_ce1/lib/python3.10/lib-dynload
   
   /home/ava_mohammadi/miniconda3/envs/annex96_ce1/lib/python3.10/site-packages

After cleanup:
   /home/ava_mohammadi/PhD/annex96_ce1/annex96_common_exercise_1/notebooks
   /home/ava_mohammadi/miniconda3/envs/annex96_ce1/lib/python310.zip
   /home/ava_mohammadi/miniconda3/envs/annex96_ce1/lib/python3.10
   /home/ava_mohammadi/miniconda3/envs/annex96_ce1/lib/python3.10/lib-dynload
   
   /home/ava_mohammadi/miniconda3/envs/annex96_ce1/lib/python3.10/site-packages


In [3]:
from pathlib import Path
from citylearn.citylearn import CityLearnEnv

# From your earlier cells:
# NOTEBOOK_DIR = Path(os.getcwd())
# BASE_DIR = NOTEBOOK_DIR.parent
# CLIMATE = "TX" or "VT"

DATASET_DIR = BASE_DIR / "data" / "datasets" / f"annex96_ce1_{CLIMATE.lower()}_neighborhood"
SCHEMA_PATH = DATASET_DIR / "schema.json"

env = CityLearnEnv(
    schema=str(SCHEMA_PATH),
    root_directory=str(DATASET_DIR),
    central_agent=True,
)


Couldn't import dot_parser, loading of dot files will not be possible.


In [4]:
from citylearn.citylearn import CityLearnEnv

env = CityLearnEnv(
    schema=str(SCHEMA_PATH),
    root_directory=str(DATASET_DIR),
    central_agent=True,  # single agent controlling all buildings
)

observations, info = env.reset()
print("Number of buildings:", len(env.buildings))
print("Action names (first 10):", env.action_names[0][:10])
print("Observation names (first 15):", env.observation_names[0][:15])


Number of buildings: 25
Action names (first 10): ['heating_device', 'heating_device', 'heating_device', 'heating_device', 'heating_device', 'heating_device', 'heating_device', 'heating_device', 'heating_device', 'heating_device']
Observation names (first 15): ['month', 'hour', 'outdoor_dry_bulb_temperature', 'direct_solar_irradiance', 'outdoor_dry_bulb_temperature_predicted_1', 'outdoor_dry_bulb_temperature_predicted_2', 'outdoor_dry_bulb_temperature_predicted_3', 'direct_solar_irradiance_predicted_1', 'direct_solar_irradiance_predicted_2', 'direct_solar_irradiance_predicted_3', 'indoor_dry_bulb_temperature', 'non_shiftable_load', 'dhw_demand', 'cooling_demand', 'heating_demand']


In [5]:
from citylearn.agents.rbc import HourRBC
from citylearn.building import Building

# Get hour indices (typically 1..24)
hours = Building.get_periodic_observation_metadata()['hour']

battery_action_map = {}

for action_name in env.action_names[0]:
    # Battery / electrical storage actions
    if 'electrical_storage' in action_name or 'storage' in action_name:
        battery_action_map[action_name] = {}

        for h in hours:
            if 9 <= h <= 12:
                value = 0.2    # charge 20% of nominal power
            elif 16 <= h <= 19:
                value = -0.2   # discharge 20% of nominal power
            else:
                value = 0.0    # idle

            battery_action_map[action_name][h] = value

    # Keep thermal devices off in this simple demo
    elif action_name in ['cooling_device', 'heating_device', 'cooling_or_heating_device']:
        battery_action_map[action_name] = {h: 0.0 for h in hours}

# Create the simple hourly RBC
battery_rbc = HourRBC(env, action_map=battery_action_map)
print("Battery RBC created.")


Battery RBC created.


In [6]:
import citylearn
print("citylearn loaded from:", citylearn.__file__)


citylearn loaded from: /home/ava_mohammadi/PhD/annex96_common_exercise_1/citylearn/__init__.py


In [8]:
import numpy as np

observations, info = env.reset()
rbc_rewards = []

while not env.terminated:
    actions = battery_rbc.predict(observations)
    observations, reward, terminated, truncated, info = env.step(actions)
    rbc_rewards.append(reward)

rbc_district_load = np.array(env.net_electricity_consumption)
print("Episode finished.")
print("Length of RBC district load:", len(rbc_district_load))


AssertionError: demand is greater than heating_device max output | timestep: 0, building: resstock-amy2018-2021-release-1-147002, outage: False, demand: 13.940381050109863,output: 11.416627055757761, difference: 2.523753994352102, check: False,

## 6. Evaluate Performance (KPIs)

These KPIs will be used for comparison across all CE1 participants.


In [ ]:
kpis = env.evaluate()
kpis = kpis.pivot(index='cost_function', columns='name', values='value')
kpis


## You're all set!

You now have:

- Installed CityLearn v2.5  
- Loaded TX or VT scenario  
- Viewed district reference loads  
- Run a baseline RBC  
- Computed comparison KPIs  

If you run into any issue, please reach out in the **Annex 96 Slack**.

Happy experimenting! 🚀
